In [ ]:
import pandas as pd

orders = [
(9423517, '2022-02-04', 9001),
(4626232, '2022-02-04', 9003),
(9423534, '2022-02-04', 9001),
(9423679, '2022-02-05', 9002),
(4626377, '2022-02-05', 9003),
(4626412, '2022-02-05', 9004),
(9423783, '2022-02-06', 9002),
(4626490, '2022-02-06', 9004)
]
df_orders = pd.DataFrame(orders, columns =['OrderNo', 'Date', 'Empno'])

details = [
(9423517, 'Jeans', 'Rip Curl', 87.0, 1),
(9423517, 'Jacket', 'The North Face', 112.0, 1),
(4626232, 'Socks', 'Vans', 15.0, 1),
(4626232, 'Jeans', 'Quiksilver', 82.0, 1),
(9423534, 'Socks', 'DC', 10.0, 2),
(9423534, 'Socks', 'Quiksilver', 12.0, 2),
(9423679, 'T-shirt', 'Patagonia', 35.0, 1),
(4626377, 'Hoody', 'Animal', 44.0, 1),
(4626377, 'Cargo Shorts', 'Animal', 38.0, 1),
(4626412, 'Shirt', 'Volcom', 78.0, 1),
(9423783, 'Boxer Shorts', 'Superdry', 30.0, 2),
(9423783, 'Shorts', 'Globe', 26.0, 1),
(4626490, 'Cargo Shorts', 'Billabong', 54.0, 1),
(4626490, 'Sweater', 'Dickies', 56.0, 1)
]
df_details = pd.DataFrame(details, columns =['OrderNo', 'Item', 'Brand', 'Price', 'Quantity'])

emps = [
(9001, 'Jeff Russell', 'LA'),
(9002, 'Jane Boorman', 'San Francisco'),
(9003, 'Tom Heints', 'NYC'),
(9004, 'Maya Silver', 'Philadelphia')
]
df_emps = pd.DataFrame(emps, columns =['Empno', 'Empname', 'Location'])

locations = [
('LA', 'West'),
('San Francisco', 'West'),
('NYC', 'East'),
('Philadelphia', 'East')
]
df_locations = pd.DataFrame(locations, columns =['Location', 'Region'])

#COMBINARE DATAFRAME
df_sales = df_orders.merge(df_details)
#print(df_sales)

df_sales['Total'] = df_sales['Price'] * df_sales['Quantity']
df_sales = df_sales[['Date','Empno','Total']]

df_sales_emps = df_sales.merge(df_emps)
df_result = df_sales_emps.merge(df_locations)
print(df_result)

#scegliere quali colonne showare
df_result = df_result[['Date','Region','Total']]


          Date  Empno  Total       Empname       Location Region
0   2022-02-04   9001   87.0  Jeff Russell             LA   West
1   2022-02-04   9001  112.0  Jeff Russell             LA   West
2   2022-02-04   9003   15.0    Tom Heints            NYC   East
3   2022-02-04   9003   82.0    Tom Heints            NYC   East
4   2022-02-04   9001   20.0  Jeff Russell             LA   West
5   2022-02-04   9001   24.0  Jeff Russell             LA   West
6   2022-02-05   9002   35.0  Jane Boorman  San Francisco   West
7   2022-02-05   9003   44.0    Tom Heints            NYC   East
8   2022-02-05   9003   38.0    Tom Heints            NYC   East
9   2022-02-05   9004   78.0   Maya Silver   Philadelphia   East
10  2022-02-06   9002   60.0  Jane Boorman  San Francisco   West
11  2022-02-06   9002   26.0  Jane Boorman  San Francisco   West
12  2022-02-06   9004   54.0   Maya Silver   Philadelphia   East
13  2022-02-06   9004   56.0   Maya Silver   Philadelphia   East


In [1]:
#raggruppare e aggregare dati pag 38
df_date_region = df_result.groupby(['Date','Region']).sum()
#print(df_date_region)
#print(df_date_region.index)

#accedere alle cifre totali per una determinata data e regione
df_date_region [df_date_region.index.isin( [('2022-02-05', 'West')])]
#è possibile passare qualsiasi indice in index.isin() in qualsiasi ordine,
df_date_region[df_date_region.index.isin([('2022-02-06', 'East'), ('2022-02-04', 'East'), ('2022-02-05', 'West')])]

#per estrarre un intervallo di valori aggregati da un DataFrame
df_date_region[('2022-02-04', 'East'):('2022-02-05', 'West')]

#Poiché in questo particolare esempio si stanno ottenendo i dati di vendita per tutte le regioni in un intervallo di date specificato, si possono omettere i nomi delle regioni e passare solo le date:
df_date_region['2022-02-04':'2022-02-05']

#ottenere i dati di vendita per uno specifico intervallo di date, selezionando tutti i contenuti del livello Region.
df_date_region.loc[(slice('2022-02-05', '2022-02-06'), slice(None)), :]

#sostituisce slice(None) con slice('East'), riducendo così i dati di vendita recuperati alle sole righe contenenti East, prese nell’intervallo di date specificato
df_date_region.loc[(slice('2022-02-05', '2022-02-06'), slice('East')), :]

#per trovare il totale delle vendite in tutte le regioni e in tutte le date.
ps = df_date_region.sum(axis = 0)
print(ps)

#Per aggiungere la nuova serie al DataFrame df_date_region è necessario innanzitutto assegnarle un nome. 
ps.name=('All','All')
#Il primo 'All' della tupla si riferisce alla componente Date della chiave di indice, mentre il secondo si riferisce alla componente Region della chiave di indice.

#Ora è possibile aggiungere la serie al DataFrame
ps_df = ps.to_frame().T  # Trasforma la serie in DataFrame con una riga
df_date_region_total = pd.concat([df_date_region, ps_df])

#È possibile accedere alla riga del totale generale in base al suo indice, come per qualsiasi altra riga di un DataFrame.
df_date_region_total[df_date_region_total.index.isin([('All', 'All')])]

#AGGIUNGERE TOTALI PARZIALI
df_totals = pd.DataFrame()
for date, date_df in df_date_region.groupby(level=0):
    df_totals = pd.concat(df_totals, date_df)
    ps = date_df.sum(axis = 0)
    ps.name=(date,'All')
    # Convertire la serie in DataFrame prima di concatenare
    ps_df = ps.to_frame().T
    df_totals = pd.concat(df_totals, ps_df)

#aggiungere riga del totale al dataframe
total_row = df_date_region_total.loc[('All','All'):('All','All')]
df_totals = pd.concat([df_totals, total_row])

NameError: name 'df_result' is not defined